In [6]:
import re
import time
import pandas as pd
import numpy as np
from urllib.parse import urljoin, quote
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ── Config ──────────────────────────────────────────────────────────────────
LISTA_PROFUMI_FILE = "lista_profumi.txt"
#cambiare qui il nome del dataset per salvarne diversi (es. profumi_dataset_guerlain, rispetto alla lista ext associata)
OUTPUT_CSV = "profumi_dataset.csv"
WAIT_SECONDS = 10

def leggi_profumi():
    """Legge la lista dei profumi dal file txt"""
    try:
        with open(LISTA_PROFUMI_FILE, 'r', encoding='utf-8') as f:
            contenuto = f.read().strip()
            if not contenuto:
                return []
            profumi = [p.strip() for p in contenuto.split(',') if p.strip()]
            return profumi
    except FileNotFoundError:
        print(f"⚠ File {LISTA_PROFUMI_FILE} non trovato. Creo un file vuoto.")
        open(LISTA_PROFUMI_FILE, 'w', encoding='utf-8').close()
        return []

def rimuovi_primo_profumo(profumi_rimanenti):
    """Rimuove il primo profumo dalla lista e riscrive il file"""
    with open(LISTA_PROFUMI_FILE, 'w', encoding='utf-8') as f:
        if profumi_rimanenti:
            f.write(', '.join(profumi_rimanenti))

def estrai_main_accords(soup):
    """Estrae i main accords e i loro punteggi dalla pagina"""
    accords = {}
    
    try:
        # Cerca tutti gli elementi che contengono style con width
        # I main accords sono tipicamente in div con class contenente "accord-bar" o simili
        # oppure cercando direttamente elementi con width% nello style
        
        # Cerca il container dei main accords
        accord_elements = soup.find_all(['div', 'span'], style=re.compile(r'width:\s*[\d.]+%'))
        
        for elem in accord_elements:
            style = elem.get('style', '')
            width_match = re.search(r'width:\s*([\d.]+)%', style)
            
            if width_match:
                width_percent = float(width_match.group(1))
                
                # Cerca il testo dell'accord (potrebbe essere nel testo dell'elemento o in un child)
                accord_name = elem.get_text(strip=True)
                
                # Se il testo è vuoto, cerca in elementi vicini o parent
                if not accord_name or len(accord_name) > 50:
                    # Prova a cercare in sibling o parent
                    parent = elem.parent
                    if parent:
                        accord_name = parent.get_text(strip=True)
                
                # Pulisci il nome dell'accord
                accord_name = accord_name.strip()
                
                # Filtra solo accords validi (nomi corti e rilevanti)
                if accord_name and len(accord_name) < 30 and width_percent > 0:
                    # Se l'accord contiene già altri testi, prendi solo la prima parte
                    accord_name_clean = accord_name.split('\n')[0].strip()
                    
                    if accord_name_clean and accord_name_clean.lower() not in ['main accords', 'notes', 'accords']:
                        accords[accord_name_clean] = round(width_percent, 2)
        
        # Metodo alternativo: cerca specificamente per la sezione main accords
        if not accords:
            # Cerca la sezione "main accords"
            main_accord_section = soup.find(text=re.compile(r'main accords', re.IGNORECASE))
            if main_accord_section:
                parent_section = main_accord_section.find_parent()
                if parent_section:
                    # Trova tutte le barre nell'area
                    bars = parent_section.find_all_next(['div', 'span'], style=re.compile(r'width:\s*[\d.]+%'), limit=15)
                    
                    for bar in bars:
                        style = bar.get('style', '')
                        width_match = re.search(r'width:\s*([\d.]+)%', style)
                        
                        if width_match:
                            width_percent = float(width_match.group(1))
                            text = bar.get_text(strip=True)
                            
                            if text and len(text) < 30 and width_percent > 0:
                                accords[text] = round(width_percent, 2)
        
        print(f"  ✓ Estratti {len(accords)} main accords")
        for acc, score in sorted(accords.items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"    - {acc}: {score}%")
            
    except Exception as e:
        print(f"  ⚠ Errore nell'estrazione degli accords: {e}")
    
    return accords

def calcola_voto_ponderato(rating, voti):
    """Calcola un voto ponderato che tiene conto del numero di voti"""
    # Formula: rating * log10(voti + 1)
    # Dà più peso ai profumi con molti voti senza far esplodere i numeri
    return round(rating * np.log10(voti + 1), 2)

def processa_profumo(driver, wait, profumo):
    """Processa un singolo profumo e ritorna i dati estratti"""
    print(f"\n{'='*60}")
    print(f"Processando: {profumo}")
    print('='*60)
    
    query_encoded = quote(profumo)
    search_url = f"https://www.fragrantica.com/search/?query={query_encoded}"
    
    try:
        # 1) Apri la pagina di ricerca
        print(f"Apertura: {search_url}")
        driver.get(search_url)
        time.sleep(2)
        
        # 2) Attendi che compaiano risultati
        print("Attendo i risultati di ricerca...")
        wait.until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/perfume/')]")))
        
        # 3) Prendi il primo link /perfume/
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")
        link = soup.find("a", href=lambda href: href and "/perfume/" in href)
        
        if not link:
            print("⚠ Nessun link trovato che contiene '/perfume/'")
            return None
        
        first_perfume_url = urljoin(driver.current_url, link["href"])
        print(f"✓ Primo profumo trovato: {first_perfume_url}")
        
        # 4) Apri la pagina del profumo
        driver.get(first_perfume_url)
        time.sleep(2)
        
        # 5) Attendi che il rating sia presente
        print("Attendo il rating...")
        wait.until(
            EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Perfume rating')]"))
        )
        
        # 6) Estrai HTML
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")
        text = soup.get_text(separator=" ", strip=True)
        
        # 7) Cerca il rating
        match = re.search(r"Perfume rating\s+([\d.]+)\s+out of 5\s+with\s+([\d,]+)\s+votes", text, re.IGNORECASE)
        
        if not match:
            print("\n⚠ Stringa con rating non trovata.")
            return None
        
        rating = float(match.group(1))
        votes_str = match.group(2).replace(",", "")
        voti = int(votes_str)
        
        print(f"\n✓ Rating: {rating}/5.0")
        print(f"✓ Voti: {voti}")
        
        # 8) Calcola voto ponderato
        voto_ponderato = calcola_voto_ponderato(rating, voti)
        print(f"✓ Voto ponderato: {voto_ponderato}")
        
        # 9) Estrai main accords
        print("\nEstrazione main accords...")
        accords = estrai_main_accords(soup)
        
        # 10) Prepara il dizionario risultato
        risultato = {
            'nome_profumo': profumo,
            'rating': rating,
            'numero_voti': voti,
            'voto_ponderato': voto_ponderato,
            'url': first_perfume_url
        }
        
        # Aggiungi gli accords come colonne separate
        for accord_name, accord_score in accords.items():
            # Normalizza il nome dell'accord per usarlo come nome colonna
            col_name = f"accord_{accord_name.lower().replace(' ', '_')}"
            risultato[col_name] = accord_score
        
        return risultato
            
    except Exception as e:
        print(f"\n❌ Errore durante il processamento: {e}")
        driver.save_screenshot(f"error_{profumo.replace(' ', '_')}.png")
        return None

# ── Main ────────────────────────────────────────────────────────────────────

# Configura le opzioni di Chrome
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

prefs = {
    "profile.default_content_setting_values.notifications": 2,
    "profile.default_content_settings.popups": 0,
    "profile.cookie_controls_mode": 0
}
chrome_options.add_experimental_option("prefs", prefs)

# Avvia il browser
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, WAIT_SECONDS)

# Lista per salvare tutti i risultati
risultati_lista = []

try:
    # Leggi la lista dei profumi
    profumi = leggi_profumi()
    
    if not profumi:
        print("⚠ Nessun profumo da processare nel file.")
        print(f"Aggiungi profumi in {LISTA_PROFUMI_FILE} separati da virgola.")
        print("Esempio: black opium, sauvage dior, chanel no 5")
    else:
        print(f"\n📋 Trovati {len(profumi)} profumi da processare")
        print(f"Profumi: {', '.join(profumi)}\n")
        
        profumi_processati = 0
        profumi_con_successo = 0
        
        while profumi:
            profumo_corrente = profumi[0]
            
            # Processa il profumo
            risultato = processa_profumo(driver, wait, profumo_corrente)
            profumi_processati += 1
            
            if risultato:
                risultati_lista.append(risultato)
                profumi_con_successo += 1
                print(f"✓ Dati salvati in memoria")
            
            # Rimuovi il profumo processato dalla lista
            profumi.pop(0)
            rimuovi_primo_profumo(profumi)
            print(f"✓ Profumo rimosso da {LISTA_PROFUMI_FILE}")
            
            # Pausa tra un profumo e l'altro
            if profumi:
                print(f"\n⏳ Pausa di 3 secondi prima del prossimo profumo...")
                print(f"Rimangono {len(profumi)} profumi da processare")
                time.sleep(3)
        
        # Crea DataFrame e salva in CSV
        if risultati_lista:
            df = pd.DataFrame(risultati_lista)
            
            # Riordina le colonne: prima quelle base, poi gli accords
            base_cols = ['nome_profumo', 'rating', 'numero_voti', 'voto_ponderato', 'url']
            accord_cols = [col for col in df.columns if col.startswith('accord_')]
            df = df[base_cols + sorted(accord_cols)]
            
            # Salva in CSV
            df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
            
            print(f"\n{'='*60}")
            print(f"✓ Tutti i profumi sono stati processati!")
            print(f"Totale processati: {profumi_processati}")
            print(f"Con successo: {profumi_con_successo}")
            print(f"Falliti: {profumi_processati - profumi_con_successo}")
            print(f"\n📊 Dataset salvato in: {OUTPUT_CSV}")
            print(f"📈 Righe: {len(df)}, Colonne: {len(df.columns)}")
            print(f"\nColonne del dataset:")
            for i, col in enumerate(df.columns, 1):
                print(f"  {i}. {col}")
            print('='*60)
            
            # Mostra anteprima
            print("\n📋 Anteprima dati (prime 3 righe, colonne base):")
            print(df[base_cols].head(3).to_string(index=False))
        else:
            print("\n⚠ Nessun dato da salvare")
        
except KeyboardInterrupt:
    print("\n\n⚠ Interruzione manuale rilevata")
    if risultati_lista:
        df = pd.DataFrame(risultati_lista)
        df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
        print(f"✓ Dati parziali salvati in {OUTPUT_CSV}")
except Exception as e:
    print(f"\n❌ Errore generale: {e}")
    driver.save_screenshot("error_general.png")
    if risultati_lista:
        df = pd.DataFrame(risultati_lista)
        df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
        print(f"✓ Dati parziali salvati in {OUTPUT_CSV}")
finally:
    driver.quit()
    print("\n✓ Browser chiuso")

⚠ Nessun profumo da processare nel file.
Aggiungi profumi in lista_profumi.txt separati da virgola.
Esempio: black opium, sauvage dior, chanel no 5

✓ Browser chiuso
